# Quantizing Llama 7B to W4A16 Using SparseML's OneShot Pathway

This example notebook walks through how to quantize Llama 7B using SparseML. We apply int4 channel-wise quantization all Linear layers, using UltraChat 200k as a calibration dataset.

This example requires at least 30GB of GPU memory to run. The memory requirement can be reduced to 16GB by setting `sequential_update: true` in the recipe definition, but this will increase the runtime significantly.

In [ ]:
import torch
from sparseml.transformers import SparseAutoModelForCausalLM, oneshot

SparseML uses recipes to define configurations for different oneshot algorithms. Recipes can be defined as a string or a yaml file. Below we create a sample recipe for GPTQ quantization. The recipe is made up of two different algorithms, called modifiers.

1. **vLLMQuantizationModifier**: calibrates the model for quantization by calculating scale and zero points from a small amount of calibration data
2. **SparseGPTModifier**: applies the GPTQ algorithm, using the result of the vLLMQuantizationModifier to determine the best quantization bin to place each linear weight into

In [ ]:
recipe="""
test_stage:
    quant_modifiers:
        vLLMQuantizationModifier:
            ignore: ["lm_head"]
            config_groups:
                group_0:
                    weights:
                        num_bits: 4
                        type: "int"
                        symmetric: false
                        strategy: "channel"
                    targets: ["Linear"]
        SparseGPTModifier:
            sparsity: 0.0
            quantize: True
            sequential_update: true
"""

Next we need to initialize the model we wish to quantize, and define a dataset for calibration. We will use a llama2 7b model that has been pretrained on the ultrachat 200k dataset. We will use the same dataset the model has been pretrained on for our one shot calibration. 

SparseML supports several datasets, such as ultrachat-200k, out of the box. You can also pass in a tokenized `datasets.Dataset` object for custom dataset support.

In [ ]:

# by setting the device_map to auto, we can spread the model evenly across all available GPUs
# load the model in as bfloat16 to save on memory and compute
model_stub = "zoo:llama2-7b-ultrachat200k_llama2_pretrain-base"
model = SparseAutoModelForCausalLM.from_pretrained(model_stub, torch_dtype=torch.bfloat16, device_map="auto")

# uses SparseML's built-in preprocessing for ultra chat
dataset = "ultrachat-200k"

# save location of quantized model
output_dir = "./output_llama7b_W4A16_channel_compressed"

Now we will configure our calibration dataset. To save on load time, we load only a small subset of ultrachat200k's `train_gen` split and label it as calibration data. For oneshot we do not need to pad the input, so we set `pad_to_max_length` to false. We also truncate each sample to a maximum of 512 tokens and select 512 samples for calibration. 

Using more calibration samples can improve model performance but will take longer to run. Generally 256-2048 calibration samples is recommended.

In [ ]:
# set dataset config parameters
splits = {"calibration": "train_gen[:5%]"}
max_seq_length = 512
pad_to_max_length = False
num_calibration_samples = 512

Finally, we can launch our quantization recipe using the `oneshot` function. This function call will apply the algorithms defined in `recipe` to the input `model`, using `num_calibration_samples` from `dataset` as calibration data. We will save the quantized model to `output_dir`.

By setting `save_compressed` to True, the model will be saved with all of the quantized weights stored as int8, enabling the model to be loaded by vLLM. Once a model has been saved in this way, you can no longer reload the model in SparseML. To save the model in a "fake quantized" state instead so that it can be reloaded in SparseML or Transformers, set `save_compressed` to False

In [ ]:
oneshot(
    model=model,
    dataset=dataset,
    recipe=recipe,
    output_dir=output_dir,
    splits=splits,
    max_seq_length=max_seq_length,
    pad_to_max_length=pad_to_max_length,
    num_calibration_samples=num_calibration_samples,
    save_compressed=True
)

The quantized model should now be stored in the defined `output_dir`. Its `config.json` will contain a new `compression_config` field that describes how the model has been quantized. This config will be used to load the model into vLLM.